In [1]:
import os
from lisa.config import PILOT_DATA_DIR
import kineticstoolkit as ktk
import polars as pl
import warnings

2024-07-22 12:42:15.868 | INFO     | lisa.config:<module>:13 - PROJ_ROOT path is: /Users/tomwilson/code/LISA


In [29]:
filename = 'Walk1_7ms 5decline02'
file = os.path.join(PILOT_DATA_DIR, filename + '.c3d')

In [24]:
file

'/Users/tomwilson/code/LISA/data/raw/Wisdish, Sam - LISA_Study1_DataCollection/LISA 1 Pilot Data/Pilot 1/120724/Walking 1_4ms01.c3d'

In [30]:
# Suppress ktk timeseries warning
warnings.filterwarnings("ignore", category=UserWarning)

# TODO check convert_point_unit doesn't affect analog data
c3d_contents = ktk.read_c3d(file, convert_point_unit=True)

In [37]:
c3d_contents['Analogs']

TimeSeries with attributes:
         time: <array of shape (30000,)>
         data: <dict with 32 entries>
    time_info: {'Unit': 's'}
    data_info: <dict with 28 entries>
       events: []

In [6]:
def c3d_to_df(c3d_contents, type='Points'):
    pd_df = c3d_contents[type].to_dataframe()
    return pl.from_pandas(pd_df, include_index=True)

point_df = c3d_to_df(c3d_contents, type='Points')

In [7]:
def process_time(df):
    # Remove t dim in points data
    columns_to_drop = [col for col in df.columns if col.endswith('[3]')]

    # Drop these columns from the DataFrame
    df = df.drop(columns_to_drop)

    # Name time column
    return df.rename(lambda col: "TIME" if "None" in col else col)

point_df = process_time(point_df)

In [8]:
point_df.is_empty()

False

In [9]:
# point_df.plot(x='TIME', y=['*34[0]', '*34[1]', '*34[2]', '*40[0]', '*40[1]', '*40[2]' , '*41[0]', '*41[1]', '*41[2]', '*42[0]', '*42[1]', '*42[2]'])

In [10]:
point_df.plot(x='TIME', y=['RASIS[2]', '*34[2]', '*37[2]'])

:NdOverlay   [Variable]
   :Curve   [TIME]   (value)

In [21]:
# Animation
%matplotlib qt5
markers = c3d_contents["Points"]
p = ktk.Player(markers)
p.up = 'z'
p.play()

In [12]:
c3d_contents["Points"].plot()

In [13]:
point_df.null_count()

TIME,RASIS[0],RASIS[1],RASIS[2],LPSIS[0],LPSIS[1],LPSIS[2],RPSIS[0],RPSIS[1],RPSIS[2],LLKN[0],LLKN[1],LLKN[2],LMKN[0],LMKN[1],LMKN[2],LT4[0],LT4[1],LT4[2],LT1[0],LT1[1],LT1[2],LT2[0],LT2[1],LT2[2],LT3[0],LT3[1],LT3[2],LLM[0],LLM[1],LLM[2],LMM[0],LMM[1],LMM[2],LSK4[0],LSK4[1],LSK4[2],…,RCALC[2],RMH5[0],RMH5[1],RMH5[2],RMH1[0],RMH1[1],RMH1[2],LASIS[0],LASIS[1],LASIS[2],*34[0],*34[1],*34[2],*35[0],*35[1],*35[2],*36[0],*36[1],*36[2],*37[0],*37[1],*37[2],*38[0],*38[1],*38[2],*39[0],*39[1],*39[2],*40[0],*40[1],*40[2],*41[0],*41[1],*41[2],*42[0],*42[1],*42[2]
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,…,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,3,3,3,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,0,0,0,…,0,0,0,0,0,0,0,129,129,129,2167,2167,2167,0,0,0,0,0,0,4525,4525,4525,0,0,0,0,0,0,5890,5890,5890,5820,5820,5820,5936,5936,5936


In [14]:
# Identify columns where all values are null
all_null_columns = [col for col in point_df.columns if point_df[col].is_null().all()]

# Print the columns where all values are null
print("Columns where all values are null:")
for col in all_null_columns:
    print(col)

Columns where all values are null:


In [15]:
# Identify columns where all values are 1.0
all_ones_columns = [col for col in point_df.columns if (point_df[col] == 1.0).all()]
print(len(all_ones_columns))
# Print the columns where all values are 1.0
print("Columns where all values are 1.0:")
for col in all_ones_columns:
    print(col)

0
Columns where all values are 1.0:


In [16]:
# Remove floor markers

# Get the count of the 'TIME' column
time_count = point_df['TIME'].count()

# Find columns that start with '*' and have the same count as the 'TIME' column
matching_columns = [col for col in point_df.columns if col.startswith('*') and point_df[col].count() == time_count]

# Remove the identified columns from the DataFrame
point_df = point_df.drop(matching_columns)

print("Dropping columns: " + ', '.join(matching_columns))

Dropping columns: *35[0], *35[1], *35[2], *36[0], *36[1], *36[2], *38[0], *38[1], *38[2], *39[0], *39[1], *39[2]


In [17]:
activity_categories = ['walk', 'jog', 'run', 'jump']

def find_activity_category(filename):
    for activity in activity_categories:
        if filename.lower().startswith(activity):
            return activity
    return None  # Return None or a default value if no match is found

point_df = point_df.with_columns(pl.lit(find_activity_category(filename)).alias('ACTIVITY'))
point_df.tail()

TIME,RASIS[0],RASIS[1],RASIS[2],LPSIS[0],LPSIS[1],LPSIS[2],RPSIS[0],RPSIS[1],RPSIS[2],LLKN[0],LLKN[1],LLKN[2],LMKN[0],LMKN[1],LMKN[2],LT4[0],LT4[1],LT4[2],LT1[0],LT1[1],LT1[2],LT2[0],LT2[1],LT2[2],LT3[0],LT3[1],LT3[2],LLM[0],LLM[1],LLM[2],LMM[0],LMM[1],LMM[2],LSK4[0],LSK4[1],LSK4[2],…,RSK4[0],RSK4[1],RSK4[2],RSK3[0],RSK3[1],RSK3[2],RSK2[0],RSK2[1],RSK2[2],RCALC[0],RCALC[1],RCALC[2],RMH5[0],RMH5[1],RMH5[2],RMH1[0],RMH1[1],RMH1[2],LASIS[0],LASIS[1],LASIS[2],*34[0],*34[1],*34[2],*37[0],*37[1],*37[2],*40[0],*40[1],*40[2],*41[0],*41[1],*41[2],*42[0],*42[1],*42[2],ACTIVITY
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
29.975,0.115971,0.38139,0.935973,-0.097947,0.235789,0.964225,0.006626,0.221889,0.955981,-0.182987,0.501043,0.445225,-0.061537,0.526013,0.424752,-0.207611,0.484186,0.653171,-0.216575,0.443147,0.642751,-0.211547,0.448459,0.598096,-0.203494,0.493706,0.608505,-0.156591,0.571855,0.029314,-0.068254,0.588515,0.040779,-0.19214,0.537592,0.311652,…,0.153927,0.02845,0.369738,0.14633,-0.006289,0.33639,0.142986,-0.030632,0.365701,0.039791,-0.25367,0.207298,0.142528,-0.156681,0.064784,0.084224,-0.098568,0.057702,-0.14246,0.430566,0.931987,0.147307,0.283514,0.859231,-0.210982,0.385403,0.847059,null,null,null,null,null,null,null,null,null,"""walk"""
29.98,0.115145,0.382832,0.933068,-0.098723,0.236467,0.961628,0.006037,0.222858,0.952882,-0.183567,0.501079,0.442849,-0.062063,0.52798,0.422415,-0.207958,0.485554,0.650176,-0.217465,0.444576,0.640205,-0.212078,0.449525,0.595416,-0.203787,0.495051,0.605687,-0.155826,0.568173,0.029799,-0.067798,0.585846,0.039806,-0.192088,0.537784,0.309583,…,0.154726,0.029929,0.369071,0.146673,-0.00544,0.336915,0.143477,-0.029187,0.366501,0.038952,-0.254415,0.214973,0.142015,-0.164947,0.067558,0.08437,-0.107149,0.058118,null,null,null,0.14657,0.285348,0.855802,-0.211388,0.387273,0.845011,null,null,null,null,null,null,null,null,null,"""walk"""
29.985,0.11426,0.384152,0.930443,-0.0996,0.237126,0.959361,0.005441,0.223692,0.950011,-0.184359,0.500524,0.440912,-0.061548,0.528972,0.420285,-0.209063,0.486724,0.647136,-0.218676,0.445613,0.637679,-0.212302,0.44964,0.592883,-0.203175,0.494561,0.602207,-0.155575,0.564383,0.029013,-0.068199,0.581325,0.039106,-0.192314,0.538896,0.309091,…,0.155767,0.032122,0.368442,0.147603,-0.003818,0.336845,0.143859,-0.026896,0.36726,0.038033,-0.254052,0.222945,0.141418,-0.17263,0.070897,0.084686,-0.11479,0.059111,-0.21213,0.388892,0.843364,0.145846,0.2871,0.852446,null,null,null,null,null,null,null,null,null,null,null,null,"""walk"""
29.99,0.113419,0.385302,0.928106,-0.100178,0.237322,0.957667,0.005032,0.224194,0.947459,-0.18576,0.500062,0.440583,-0.060821,0.528443,0.419772,-0.21135,0.487359,0.644311,-0.219777,0.445895,0.635526,-0.21223,0.449058,0.590756,-0.204335,0.494405,0.599347,-0.1561,0.559674,0.027324,-0.067846,0.575238,0.039415,-0.192412,0.540746,0.310192,…,0.15657,0.035229,0.36763,0.148228,-0.001464,0.33694,0.144246,-0.023771,0.367876,0.037131,-0.252833,0.231159,0.140627,-0.179758,0.07456,0.084883,-0.121838,0.060113,-0.213135,0.390181,0.842149,0.145191,0.288825,0.849392,null,null,null,null,null,null,null,null,null,null,null,null,"""walk"""
29.995,0.11254,0.386239,0.926011,-0.100564,0.237333,0.956682,0.004648,0.224687,0.945235,-0.186827,0.500911,0.441738,-0.059593,0.527183,0.420494,-0.214477,0.487803,0.642354,-0.220649,0.445916,0.63421,-0.212775,0.448338,0.589556,-0.207173,0.494375,0.597414,-0.156227,0.552177,0.028659,-0.067947,0.56752,0.038831,-0.192478,0.542297,0.311486,…,0.157386,0.039126,0.366821,0.148866,0.001479,0.337075,0.144821,-0.019844,0.368597,0.036139,-0.250457,0.239494,0.139981,-0.186247,0.078638,0.085467,-0.12846,0.061382,-0.214558,0.391174,0.841677,0.144612,0.290458,0.846498,null,null,null,null,null,null,null,null,nul

In [32]:
# Analog Data
analog_df = c3d_to_df(c3d_contents, 'Analogs')

analog_df = process_time(analog_df)

In [43]:
analog_df.describe()

statistic,TIME,Force.Fx1,Force.Fy1,Force.Fz1,Moment.Mx1,Moment.My1,Moment.Mz1,Force.Fx2,Force.Fy2,Force.Fz2,Moment.Mx2,Moment.My2,Moment.Mz2,Angle.Pitch,Length.Sway,Velocity.Left Belt Speed,Velocity.Right Belt Speed,Electric Potential.1,Global Angle.x,Global Angle.y,Global Angle.z,HighG.x,HighG.y,HighG.z,accel.x,accel.y,accel.z,gyro.x,gyro.y,gyro.z,mag.x,mag.y,mag.z
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0,30000.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",14.9995,33.384158,-152.986713,-375.986494,-23087.54721,-35085.029692,35063.911243,-31.322025,-153.749619,-363.930821,-14153.964427,48565.026597,-37308.032994,0.087197,-0.001378,-1.702804,-1.711119,2.428828,-0.893309,-1.31799,-0.40006,12.908143,1.640724,1.783287,10.524194,-4.059774,-2.114209,-0.078882,-0.137276,-0.011308,-0.000032,-0.000032,-0.000016
"""std""",8.660398,47.456131,134.511076,423.781302,174144.555851,41266.838559,21198.792594,43.410619,120.490926,406.86717,155442.817288,57269.875497,21865.25381,0.000115,0.000765,0.001251,0.001277,0.67279,0.256936,0.144253,0.436746,9.174376,10.920668,7.911796,9.741091,11.516548,8.462685,2.311347,1.330682,4.045349,0.000009,0.00001,0.000003
"""min""",0.0,-78.065254,-731.016052,-1762.883545,-844940.3125,-172326.09375,-79249.703125,-215.315582,-600.11322,-1428.013672,-720319.625,-43309.257812,-151607.75,0.087009,-0.003771,-1.706789,-1.715093,1.555537,-1.434114,-1.608125,-1.142324,-8.132344,-58.361526,-30.13751,-9.854534,-105.517258,-37.967251,-8.483789,-7.081185,-8.383679,-0.000045,-0.000047,-0.000022
"""25%""",7.5,-0.870223,-161.183624,-625.754883,-51724.144531,-61321.1875,23769.203125,-48.0051,-169.210388,-573.188782,-49947.304688,1241.270752,-46183.210938,0.087127,-0.00194,-1.703737,-1.712041,1.828364,-1.032855,-1.429652,-0.766874,7.988832,-5.286024,-1.43512,5.449682,-11.513237,-5.96252,-1.34435,-0.954932,-3.909562,-0.000038,-0.000041,-0.000018
"""50%""",15.0,23.634926,-120.647194,-293.727448,8804.609375,-24936.685547,34464.324219,-20.643524,-125.399384,-335.114227,12018.460938,38829.820312,-35221.882812,0.087164,-0.001329,-1.702821,-1.711125,2.097224,-0.951382,-1.335006,-0.476277,12.246353,2.391866,1.530794,9.978075,-2.903688,-2.050873,0.544002,-0.209911,1.292058,-0.000035,-0.000032,-0.000016
"""75%""",22.499,58.069031,-94.51458,-4.797376,59405.355469,-1114.992676,44912.464844,-1.863258,-97.957108,-2.624415,64434.671875,78099.007812,-24783.240234,0.087244,-0.000719,-1.701906,-1.710057,3.156794,-0.823879,-1.222862,-0.02953,15.16443,9.041253,4.424952,12.674904,3.595134,0.335188,1.339237,0.282864,3.200325,-0.000026,-0.000025,-0.000014
"""max""",29.999,206.016388,97.526314,94.531311,331828.09375,45588.484375,128885.171875,89.263084,95.10836,106.376007,321694.0,278954.53125,63151.070312,0.087595,0.001112,-1.698702,-1.706853,3.697565,-0.028768,-0.072225,0.452882,143.033585,75.582962,240.621704,156.901611,119.825005,130.867065,12.184665,4.557135,5.54226,-0.000002,-8.5500e-7,-6.6750e-7


In [20]:
# Join on matching timestamps
point_df.join(analog_df, on='TIME')

TIME,RASIS[0],RASIS[1],RASIS[2],LPSIS[0],LPSIS[1],LPSIS[2],RPSIS[0],RPSIS[1],RPSIS[2],LLKN[0],LLKN[1],LLKN[2],LMKN[0],LMKN[1],LMKN[2],LT4[0],LT4[1],LT4[2],LT1[0],LT1[1],LT1[2],LT2[0],LT2[1],LT2[2],LT3[0],LT3[1],LT3[2],LLM[0],LLM[1],LLM[2],LMM[0],LMM[1],LMM[2],LSK4[0],LSK4[1],LSK4[2],…,*41[2],*42[0],*42[1],*42[2],ACTIVITY,Force.Fx1,Force.Fy1,Force.Fz1,Moment.Mx1,Moment.My1,Moment.Mz1,Force.Fx2,Force.Fy2,Force.Fz2,Moment.Mx2,Moment.My2,Moment.Mz2,Angle.Pitch,Length.Sway,Velocity.Left Belt Speed,Velocity.Right Belt Speed,Electric Potential.1,Global Angle.x,Global Angle.y,Global Angle.z,HighG.x,HighG.y,HighG.z,accel.x,accel.y,accel.z,gyro.x,gyro.y,gyro.z,mag.x,mag.y,mag.z
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.11687,0.052211,1.011285,-0.058818,-0.124478,1.043918,0.046019,-0.123357,1.020504,-0.169058,-0.170044,0.521349,-0.046874,-0.149174,0.485763,-0.198806,-0.06316,0.700852,-0.199652,-0.102817,0.717995,-0.194537,-0.125516,0.679364,-0.193945,-0.082802,0.659023,-0.13262,-0.353778,0.135608,-0.046265,-0.331493,0.13522,-0.178658,-0.220675,0.390667,…,null,null,null,null,"""walk""",6.528756,-91.64798,2.936042,-4109.524902,6912.517578,24668.763672,-48.083618,-93.193214,-527.192444,101829.046875,67925.90625,-37220.324219,0.087164,-0.002245,-1.701906,-1.710057,1.840266,-0.823135,-1.447943,-0.270271,11.95933,13.394449,4.305358,10.448297,7.469909,-0.368707,1.20345,-0.120345,1.89783,-0.000037,-0.000032,-0.000018
0.005,0.116933,0.053743,1.009175,-0.057159,-0.124106,1.042485,0.047901,-0.123199,1.018471,-0.168631,-0.175453,0.521933,-0.046739,-0.154131,0.485563,-0.198649,-0.066872,0.700477,-0.199163,-0.106379,0.717992,-0.193917,-0.129449,0.679623,-0.19368,-0.086905,0.658816,-0.13239,-0.362851,0.137685,-0.046146,-0.34038,0.137049,-0.178374,-0.227388,0.391691,…,null,null,null,null,"""walk""",1.115396,-92.93515,-5.776045,-18577.65625,-3299.269775,27846.941406,-51.637466,-65.968704,-556.365112,105257.117188,72322.460938,-28604.482422,0.087175,0.000502,-1.703584,-1.710515,1.816767,-0.82814,-1.443,-0.256443,12.916076,12.676889,3.707392,11.004949,6.749254,-0.393846,1.085235,-0.132592,2.075152,-0.000037,-0.000033,-0.000017
0.01,0.116971,0.055333,1.007031,-0.055477,-0.123684,1.040882,0.049539,-0.121951,1.016533,-0.168144,-0.180602,0.52242,-0.046573,-0.15885,0.485311,-0.198463,-0.070363,0.699866,-0.198985,-0.109617,0.717924,-0.193311,-0.133109,0.679699,-0.193386,-0.090784,0.658484,-0.132129,-0.371926,0.139835,-0.045835,-0.349312,0.138918,-0.178012,-0.233805,0.392636,…,null,null,null,null,"""walk""",-4.618952,-116.083069,1.594693,-19034.611328,-4402.008789,34305.824219,-46.579582,-53.796627,-533.958923,118719.671875,67957.109375,-18018.537109,0.087164,-0.001635,-1.701753,-1.708379,1.809748,-0.833109,-1.438012,-0.242622,13.872822,11.95933,3.109426,11.5616,6.0286,-0.418985,0.96702,-0.14484,2.252475,-0.000037,-0.000034,-0.000016
0.015,0.116971,0.056952,1.004813,-0.053826,-0.123271,1.039191,0.051025,-0.119986,1.014592,-0.16775,-0.185375,0.522685,-0.046381,-0.163326,0.485026,-0.198234,-0.073594,0.699116,-0.19857,-0.112617,0.717619,-0.19271,-0.13648,0.679616,-0.193069,-0.09434,0.658008,-0.131832,-0.380933,0.142148,-0.045543,-0.358138,0.140834,-0.177182,-0.240178,0.393149,…,null,null,null,null,"""walk""",-11.199624,-136.352997,-7.225827,-19388.96875,142.827026,45576.074219,-52.147915,-64.919113,-547.95282,123307.953125,65428.960938,-28816.347656,0.087175,-0.001024,-1.704042,-1.710057,1.800593,-0.838044,-1.432978,-0.228808,14.829569,11.24177,2.511459,12.118252,5.307945,-0.444124,0.848805,-0.157087,2.429797,-0.000037,-0.000034,-0.000015
0.02,0.116922,0.05862,1.002475,-0.052127,-0.122788,1.037356,0.052698,-0.119417,1.012415,-0.167307,-0.189817,0.522904,-0.046197,-0.167603,0.484684,-0.198275,-0.076514,0.69776